In [1]:
import pandas as pd
import chainladder as cl

In [2]:
wiki = pd.read_csv('wikipedia.csv', sep=';')[['AccidentYear', 'DevelopmentYear', 'Reported', 'Paid']]

In [3]:
xyz_tri = cl.Triangle(
    data=wiki,
    origin="AccidentYear",
    development="DevelopmentYear",
    columns=["Reported", "Paid"],
    cumulative=True,
)
xyz_tri

,Triangle Summary
Valuation:,2007-12
Grain:,OYDY
Shape:,"(1, 2, 10, 10)"
Index:,[Total]
Columns:,"[Reported, Paid]"


### Age-to-age factors

In [4]:
xyz_tri['Reported'].age_to_age.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120
1998,1.166,1.056,1.027,1.012,1.004,1.002,1.001,1.001,1.000
1999,1.182,1.062,1.027,1.010,1.004,1.003,1.002,1.000,nan
2000,1.200,1.061,1.027,1.010,1.005,1.003,1.002,nan,nan
2001,1.193,1.062,1.027,1.014,1.005,1.003,nan,nan,nan
2002,1.184,1.059,1.029,1.011,1.004,nan,nan,nan,nan
2003,1.162,1.057,1.028,1.010,nan,nan,nan,nan,nan
2004,1.159,1.055,1.026,nan,nan,nan,nan,nan,nan
2005,1.160,1.056,nan,nan,nan,nan,nan,nan,nan
2006,1.173,nan,nan,nan,nan,nan,nan,nan,nan


### Simple average last 5 years

In [5]:
cl.Development(average="simple", n_periods=5).fit(xyz_tri["Reported"]).ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120
(All),1.168,1.058,1.027,1.011,1.004,1.003,1.002,1.001,1.000


### Simple average last 3 years

In [6]:
cl.Development(average="simple", n_periods=3).fit(xyz_tri["Reported"]).ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120
(All),1.164,1.056,1.027,1.012,1.005,1.003,1.002,1.001,1.000


### Volume weighted last 5 years

In [7]:
cl.Development(average="volume", n_periods=5).fit(xyz_tri["Reported"]).ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120
(All),1.168,1.058,1.027,1.011,1.004,1.003,1.002,1.001,1.000


### Volume weighted last 5 years

In [8]:
cl.Development(average="volume", n_periods=3).fit(xyz_tri["Reported"]).ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120
(All),1.164,1.056,1.027,1.012,1.005,1.003,1.002,1.001,1.000


### Cumulative to ultimate

In [9]:
cl.Development(average="volume", n_periods=3).fit(xyz_tri["Reported"]).cdf_

,12-Ult,24-Ult,36-Ult,48-Ult,60-Ult,72-Ult,84-Ult,96-Ult,108-Ult
(All),1.2900,1.1081,1.0495,1.0215,1.0099,1.0053,1.0025,1.0009,1.0004


### Estimation of ultimate reported claims

In [21]:
xyz_dev = cl.Pipeline(
    [("dev", cl.Development(average="volume", n_periods=3))]
).fit_transform(xyz_tri['Reported'])
xyz_model = cl.Chainladder().fit(xyz_dev)
xyz_model.cdf_.to_frame().style.format(precision=3)

,12-Ult,24-Ult,36-Ult,48-Ult,60-Ult,72-Ult,84-Ult,96-Ult,108-Ult,120-Ult,132-Ult
(All),1.290,1.108,1.049,1.022,1.010,1.005,1.003,1.001,1.000,1.000,1.000


In [19]:
xyz_model.ultimate_

,2261
1998,"47,742,304"
1999,"51,204,671"
2000,"54,889,953"
2001,"56,443,034"
2002,"58,903,567"
2003,"58,135,158"
2004,"58,202,908"
2005,"59,595,283"
2006,"60,548,360"
2007,"63,020,716"


#### Estimation of ultimate claims (from Wikipedia)
|Accident year	| Reported claims	| Development factor to ultimate	| Projected ultimate claims |
|---|---|---|---|
|1998	| 47,742,304	| 1.000	| 47,742,304 |
|1999	| 51,185,767	| 1.000	| 51,185,767 |
|2000	| 54,837,929	| 1.001	| 54,892,767 |
|2001	| 56,299,562	| 1.003	| 56,468,461 |
|2002	| 58,592,712	| 1.006	| 58,944,268 |
|2003	| 57,565,344	| 1.011	| 58,198,563 | 
|2004	| 56,976,657	| 1.023	| 58,287,120 |
|2005	| 56,786,410	| 1.051	| 59,682,517 |
|2006	| 54,641,339	| 1.110	| 60,651,886 |
|2007	| 48,853,563	| 1.292	| 63,118,803 |
|Total	| 543,481,587	| 		| 569,172,456 |

### Estimation of ultimate paid claims
(in progress...)

In [28]:
# Without tail factor
xyz_dev_paid = cl.Pipeline(
    [("dev", cl.Development(average="volume", n_periods=3))]
).fit_transform(xyz_tri['Paid'])
xyz_model_paid = cl.Chainladder().fit(xyz_dev_paid)
xyz_model_paid.ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120,120-132,132-144
(All),1.702,1.186,1.091,1.044,1.019,1.009,1.005,1.002,1.002,1.000,1.000


In [40]:
# With constant tail factor
xyz_dev_paid = cl.Pipeline([
    ("dev", cl.Development(average="volume", n_periods=3)),
    ("tail", cl.TailConstant(1.002))
]
).fit_transform(xyz_tri['Paid'])
xyz_model_paid = cl.Chainladder().fit(xyz_dev_paid)
xyz_model_paid.ldf_.to_frame().style.format(precision=3)

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120,120-132,132-144
(All),1.702,1.186,1.091,1.044,1.019,1.009,1.005,1.002,1.002,1.001,1.001


In [42]:
xyz_model_paid.cdf_.to_frame().style.format(precision=3)

,12-Ult,24-Ult,36-Ult,48-Ult,60-Ult,72-Ult,84-Ult,96-Ult,108-Ult,120-Ult,132-Ult
(All),2.391,1.404,1.184,1.085,1.040,1.021,1.011,1.006,1.004,1.002,1.001


In [77]:
xyz_paid = xyz_model_paid.ultimate_.to_frame()
xyz_paid.index = [x.year for x in xyz_paid.index]
xyz_paid.style.format('{:,.0f}')

,2261
1998,"47,739,475"
1999,"51,197,884"
2000,"54,872,471"
2001,"56,494,910"
2002,"58,998,156"
2003,"58,153,510"
2004,"58,363,614"
2005,"59,973,948"
2006,"61,243,882"
2007,"65,097,578"


In [78]:
wiki_paid = pd.DataFrame([47739475, 51204536, 54860424, 56493084, 58963359, 58167880, 58345519, 59963673, 61223522, 65079626], index=range(1998,2008))
wiki_paid.style.format('{:,.0f}')

,0
1998,"47,739,475"
1999,"51,204,536"
2000,"54,860,424"
2001,"56,493,084"
2002,"58,963,359"
2003,"58,167,880"
2004,"58,345,519"
2005,"59,963,673"
2006,"61,223,522"
2007,"65,079,626"


In [75]:
diff = (xyz_paid['2261']-wiki_paid[0])/wiki_paid[0]
diff.reset_index().style.format({0:'{:.3%}'})

,index,0
0,1998,0.000%
1,1999,-0.013%
2,2000,0.022%
3,2001,0.003%
4,2002,0.059%
5,2003,-0.025%
6,2004,0.031%
7,2005,0.017%
8,2006,0.033%
9,2007,0.028%
